In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from numpy import linalg as la
import sys
import matplotlib.pyplot as plt
import seaborn as sas

In [4]:
def PCA(posture):
    df = pd.read_csv(posture, sep="\t", header=None)
    #la stessa posizione viene misurata 82 volte DC
    df.drop(columns=[2048], inplace = True) #tolgo colonna
    df.drop(0, inplace = True) #tolgo riga
    df.drop(1, inplace = True) #tolgo riga
    df.drop(2, inplace = True) #tolgo riga
    
    array = df.to_numpy()
    avg = np.mean(array, axis = 1, keepdims=True)

    print(avg.shape)
    array = array - avg

    CX = np.cov(array)
    w,v = la.eig(CX) 

    CY =  np.diag(np.round(np.real(w),2))

    P = v[:,0:1]
    print(P.shape)
    P = abs(P)

    array_final = P.T.dot(array)
    
    array_final[0] = array_final[0]
    array_final[0][array_final[0] < 0] = 0
    array_final[0][array_final[0] > 3000] = 1000

    OldRange = array_final[0].max()  
    NewRange = 1000 
    for i in range(0,len(array_final[0])):
        array_final[0][i] = (((array_final[0][i]) * NewRange) / OldRange) 
        
    return array_final[0]

In [7]:
rootdir = Path("dataset/experiment-i/")
import os

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(os.path.join(subdir, file))
        file_name = os.path.join(subdir, file)

dataset/experiment-i/subject-info-i.csv
dataset/experiment-i/S11/4.txt
dataset/experiment-i/S11/12.txt
dataset/experiment-i/S11/3.txt
dataset/experiment-i/S11/9.txt
dataset/experiment-i/S11/15.txt
dataset/experiment-i/S11/7.txt
dataset/experiment-i/S11/11.txt
dataset/experiment-i/S11/16.txt
dataset/experiment-i/S11/2.txt
dataset/experiment-i/S11/17.txt
dataset/experiment-i/S11/14.txt
dataset/experiment-i/S11/1.txt
dataset/experiment-i/S11/6.txt
dataset/experiment-i/S11/8.txt
dataset/experiment-i/S11/13.txt
dataset/experiment-i/S11/10.txt
dataset/experiment-i/S11/5.txt
dataset/experiment-i/S10/4.txt
dataset/experiment-i/S10/12.txt
dataset/experiment-i/S10/3.txt
dataset/experiment-i/S10/9.txt
dataset/experiment-i/S10/15.txt
dataset/experiment-i/S10/7.txt
dataset/experiment-i/S10/11.txt
dataset/experiment-i/S10/16.txt
dataset/experiment-i/S10/2.txt
dataset/experiment-i/S10/17.txt
dataset/experiment-i/S10/14.txt
dataset/experiment-i/S10/1.txt
dataset/experiment-i/S10/6.txt
dataset/experime